In [1]:
import pandas as pd
import shutil
import os

# Load metadata
df = pd.read_csv('Downloads/data/HAM10000_metadata.csv')

# Define directories
image_folder = 'Downloads/data/images/'
train_folder = 'Downloads/data/train'

# Define class labels
CLASS_LABELS = df['dx'].unique().tolist()

# Create folders for each class
for label in CLASS_LABELS:
    os.makedirs(os.path.join(train_folder, label), exist_ok=True)

# Move images into corresponding class folders
missing_files = []
for _, row in df.iterrows():
    filename = row['image_id'] + '.jpg'
    src = os.path.join(image_folder, filename)
    dst = os.path.join(train_folder, row['dx'], filename)
    
    # Check if the image file exists
    if os.path.exists(src):
        shutil.move(src, dst)
    else:
        missing_files.append(filename)

# Report missing files
if missing_files:
    print(f"{len(missing_files)} files were not found and could not be moved.")
    #print("List of missing files:")
    #for file in missing_files:
        #print(file)

print("Images successfully organized into class folders!")


10015 files were not found and could not be moved.
Images successfully organized into class folders!


In [ ]:
from sklearn.model_selection import train_test_split
import os

# Create validation directories
for label in CLASS_LABELS:
    os.makedirs(f'Downloads/data/val/{label}', exist_ok=True)

# Split images into train/val
for label in CLASS_LABELS:
    image_files = os.listdir(f"Downloads/data/train/{label}")
    train_files, val_files = train_test_split(image_files, test_size=0.2, random_state=42)

    # Move validation images
    for file in val_files:
        shutil.move(f"Downloads/data/train/{label}/{file}", f"Downloads/data/val/{label}/{file}")

print("Train/Validation split completed!")

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define image size and batch size
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Define paths
TRAIN_DIR = "Downloads/data/train"
VAL_DIR = "Downloads/data/val"

# Data augmentation & normalization
datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,   # Flip horizontally
    fill_mode='nearest'
)

# Load train & validation data
train_data = datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)
datagen1 = ImageDataGenerator(
    rescale=1.0/255.0) 

val_data = datagen.flow_from_directory(
    VAL_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)


Found 8010 images belonging to 7 classes.
Found 2005 images belonging to 7 classes.


In [ ]:
base_model = tf.keras.applications.MobileNetV2(
    weights='imagenet', include_top=False, input_shape=(224, 224, 3)
)

# Freeze base model layers
base_model.trainable = False

# Add custom classification layers
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(516, activation='relu'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(train_data.num_classes, activation='softmax')  # Multi-class classification
])

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Print model summary
model.summary()

In [ ]:
hist=model.fit(train_data,validation_data=val_data,epochs=10 )


In [ ]:
base_model.trainable=True
model.compile(  optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy'])
hist_fine=model.fit(train_data,validation_data=val_data,epochs=5 )

In [ ]:
import tensorflow as tf

# Enable GPU growth to avoid memory issues
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ Using GPU for training")
    except RuntimeError as e:
        print(e)
else:
    print("❌ No GPU found. Using CPU.")


In [ ]:
import tensorflow as tf

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


In [ ]:
import tensorflow as tf
print(tf.__version__)


In [ ]:
import tensorflow as tf
print("TensorFlow Version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))
print(tf.test.is_built_with_cuda())  
print(tf.test.is_built_with_gpu_support())  


In [ ]:
!pip install vit-keras tensorflow

In [3]:
from vit_keras import vit
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model

num_classes = 7  # Define number of classes

# Load pre-trained ViT model
base_model = vit.vit_b16(
    image_size=224,
    pretrained=True,
    include_top=False,
    pretrained_top=False
)
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-5:]:  
    layer.trainable = True


# Define custom classifier
x = Dense(1024, activation='relu')(base_model.output)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='softmax')(x)

# Create model
model = Model(inputs=base_model.input, outputs=x)

# Print model summary
model.summary()


C:\Users\shrey\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
C:\Users\shrey\AppData\Local\Programs\Python\Python310\lib\site-packages\vit_keras\utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 14, 14
  warnings.warn(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 14, 14, 768)       590592    
                                                                 
 reshape (Reshape)           (None, 196, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 197, 768)          768       
                                                                 
 Transformer/posembed_input  (None, 197, 768)          151296    
  (AddPositionEmbs)                                              
                                                                 
 Transformer/encoderblock_0  ((None, 197, 768),        7087872   
  (TransformerBlock)          (None, 12, None, None)         

In [ ]:
!pip install tensorflow-addons
!pip install cv2

In [ ]:
!pip uninstall python


In [ ]:
pip install opencv-python


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history_tran = model.fit(train_data, validation_data=val_data, epochs=5)

In [ ]:
!pip install Pillow

In [ ]:
model.evaluate(val_data)

In [ ]:
save_path = "C:/Users/shrey/Downloads/vit_model.keras"
model.save(save_path)


In [6]:
model1 = keras.models.load_model("C:/Users/shrey/Downloads/vit_model.keras", safe_mode=False)

In [5]:
import keras

In [ ]:
model1.summary()

In [ ]:
model1.evaluate(val_data)

In [9]:
model1.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-5),loss='categorical_crossentropy',metrics=['Accuracy'])
model1.fit(train_data,validation_data=val_data,epochs=2,callbacks=[lr_scheduler])

Epoch 1/2
251/251 [==============================] - 9824s 39s/step - loss: 0.3652 - Accuracy: 0.8693 - val_loss: 0.4352 - val_Accuracy: 0.8489 - lr: 1.0000e-05
Epoch 2/2
251/251 [==============================] - 7608s 30s/step - loss: 0.3009 - Accuracy: 0.8901 - val_loss: 0.4350 - val_Accuracy: 0.8539 - lr: 1.0000e-05


In [7]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(
    monitor="val_loss", patience=3, factor=0.5, min_lr=1e-6
)


In [8]:
# Unfreeze the last 10 layers for fine-tuning
for layer in model1.layers[-15:]:
    layer.trainable = True

model1.trainable=True
model1.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 embedding (Conv2D)          (None, 14, 14, 768)       590592    
                                                                 
 reshape (Reshape)           (None, 196, 768)          0         
                                                                 
 class_token (ClassToken)    (None, 197, 768)          768       
                                                                 
 Transformer/posembed_input  (None, 197, 768)          151296    
  (AddPositionEmbs)                                              
                                                                 
 Transformer/encoderblock_0  ((None, 197, 768),        7087872   
  (TransformerBlock)          (None, 12, None, None)         

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=30,       # Rotate images by 30 degrees
    width_shift_range=0.2,   # Shift width by 20%
    height_shift_range=0.2,  # Shift height by 20%
    shear_range=0.2,         # Shear intensity
    zoom_range=0.2,          # Zoom in/out by 20%
    horizontal_flip=True,    # Flip horizontally
    fill_mode='nearest'      # Fill missing pixels
)

# Apply augmentation during training
train_generator = datagen.flow(train_data, CLASS_LABELS, batch_size=8)


In [ ]:
import tensorflow as tf

# Convert DirectoryIterator to tf.data.Dataset
train_data = tf.data.Dataset.from_generator(
    lambda: train_data, 
    output_signature=(
        tf.TensorSpec(shape=(224, 224, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(), dtype=tf.int32)  # Adjust shape based on your labels
    )
)

# Augment function
def augment(image, label):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_brightness(image, max_delta=0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    return image, label

# Apply augmentation
train_data = train_data.map(augment)


In [10]:
model1.save("C:/Users/shrey/Downloads/vit_model2.keras")

In [11]:
model1.evaluate(val_data)

63/63 [==============================] - 278s 4s/step - loss: 0.4438 - Accuracy: 0.8584


[0.44381365180015564, 0.8583540916442871]